In [2]:
import sys
sys.path.append('/media/gpt4-pdf-chatbot-langchain/langchain-ChatGLM/')
from langchain.llms.base import LLM
import torch
import transformers 
import models.shared as shared 
from abc import ABC

from langchain.llms.base import LLM
import random
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList
from typing import Optional, List, Dict, Any
from models.loader import LoaderCheckPoint
from models.extensions.callback import (Iteratorize, Stream, FixedLengthQueue) 
from models.base import (BaseAnswer,
                         AnswerResult,
                         AnswerResultStream,
                         AnswerResultQueueSentinelTokenListenerQueue)
from langchain.callbacks.manager import (
    CallbackManagerForLLMRun
)



In [3]:
import asyncio
from argparse import Namespace
from models.loader.args import parser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
 
args = parser.parse_args(args=['--model-dir', '/media/checkpoint/',  '--model', 'vicuna-13b-hf', '--no-remote-model', '--load-in-8bit'])

args_dict = vars(args)

shared.loaderCheckPoint = LoaderCheckPoint(args_dict)
torch.cuda.empty_cache()
shared.loaderCheckPoint.unload_model()
shared.loaderCheckPoint.reload_model() 


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading vicuna-13b-hf...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded the model in 182.64 seconds.


In [4]:
from abc import ABC

from langchain.llms.base import LLM
import random
import torch
import transformers
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList
from typing import Optional, List, Dict, Any
from models.loader import LoaderCheckPoint
from models.extensions.callback import (Iteratorize, Stream, FixedLengthQueue)
import models.shared as shared
from models.base import (BaseAnswer,
                         AnswerResult,
                         AnswerResultStream,
                         AnswerResultQueueSentinelTokenListenerQueue)
from langchain.callbacks.manager import (
    CallbackManagerForLLMRun
)


def _streaming_response_template() -> Dict[str, Any]:
    """
    :return: 响应结构
    """
    return {
        "text": ""
    }


def _update_response(response: Dict[str, Any], stream_response: str) -> None:
    """Update response from the stream response."""
    response["text"] += stream_response


class InvalidScoreLogitsProcessor(LogitsProcessor):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        if torch.isnan(scores).any() or torch.isinf(scores).any():
            scores.zero_()
            scores[..., 5] = 5e4
        return scores


class LLamaLLM(BaseAnswer, LLM, ABC):
    checkPoint: LoaderCheckPoint = None
    history = []
    history_len: int = 3
    max_new_tokens: int = 500
    num_beams: int = 1
    temperature: float = 0.5
    top_p: float = 0.4
    top_k: int = 10
    repetition_penalty: float = 1.12
    encoder_repetition_penalty: int = 1
    min_length: int = 0
    logits_processor: LogitsProcessorList = None
    stopping_criteria: Optional[StoppingCriteriaList] = None

    state: object = {'max_new_tokens': 50,
                     'seed': 1,
                     'temperature': 0, 'top_p': 0.1,
                     'top_k': 40, 'typical_p': 1,
                     'repetition_penalty': 1.18,
                     'encoder_repetition_penalty': 1,
                     'no_repeat_ngram_size': 0,
                     'min_length': 0,
                     'penalty_alpha': 0,
                     'num_beams': 1,
                     'length_penalty': 1,
                     'early_stopping': False, 'add_bos_token': True, 'ban_eos_token': False,
                     'truncation_length': 2048, 'custom_stopping_strings': '',
                     'cpu_memory': 0, 'auto_devices': False, 'disk': False, 'cpu': False, 'bf16': False,
                     'load_in_8bit': False, 'wbits': 'None', 'groupsize': 'None', 'model_type': 'None',
                     'pre_layer': 0, 'gpu_memory_0': 0}

    def __init__(self, checkPoint: LoaderCheckPoint = None):
        super().__init__()
        self.checkPoint = checkPoint

    @property
    def _llm_type(self) -> str:
        return "LLamaLLM"

    @property
    def _check_point(self) -> LoaderCheckPoint:
        return self.checkPoint

    def encode(self, prompt, add_special_tokens=True, add_bos_token=True, truncation_length=None):
        input_ids = self.checkPoint.tokenizer.encode(str(prompt), return_tensors='pt',
                                                     add_special_tokens=add_special_tokens)
        # This is a hack for making replies more creative.
        if not add_bos_token and input_ids[0][0] == self.checkPoint.tokenizer.bos_token_id:
            input_ids = input_ids[:, 1:]

        # Llama adds this extra token when the first character is '\n', and this
        # compromises the stopping criteria, so we just remove it
        if type(self.checkPoint.tokenizer) is transformers.LlamaTokenizer and input_ids[0][0] == 29871:
            input_ids = input_ids[:, 1:]

        # Handling truncation
        if truncation_length is not None:
            input_ids = input_ids[:, -truncation_length:]

        return input_ids.cuda()

    def decode(self, output_ids):
        reply = self.checkPoint.tokenizer.decode(output_ids, skip_special_tokens=True)
        return reply

    def generate_with_callback(self, callback=None, **kwargs):
        self.checkPoint.clear_torch_cache()
        kwargs['stopping_criteria'].append(Stream(callback_func=callback))
        with torch.no_grad():
            self.checkPoint.model.generate(**kwargs)
            print("方法结束")

    def generate_with_streaming(self, **kwargs):
        return Iteratorize(self.generate_with_callback, kwargs)

    # 将历史对话数组转换为文本格式
    def history_to_text(self, query):
        formatted_history = ''
        history = self.history[-self.history_len:] if self.history_len > 0 else []
        for i, (old_query, response) in enumerate(history):
            formatted_history += "[Round {}]\n问：{}\n答：{}\n".format(i, old_query, response)
        formatted_history += "[Round {}]\n问：{}\n答：".format(len(history), query)
        return formatted_history

    def prepare_inputs_for_generation(self,
                                      input_ids: torch.LongTensor):
        """
        预生成注意力掩码和 输入序列中每个位置的索引的张量
        # TODO 没有思路
        :return:
        """

        mask_positions = torch.zeros((1, input_ids.shape[1]), dtype=input_ids.dtype).to(self.checkPoint.model.device)

        attention_mask = self.get_masks(input_ids, input_ids.device)

        position_ids = self.get_position_ids(
            input_ids,
            device=input_ids.device,
            mask_positions=mask_positions
        )

        return input_ids, position_ids, attention_mask

    def get_position_ids(self, input_ids: torch.LongTensor, mask_positions, device):
        """
        注意力偏移量
        :param input_ids:
        :param mask_positions:
        :param device:
        :param use_gmasks:
        :return:
        """
        batch_size, seq_length = input_ids.shape
        context_lengths = [seq.tolist().index(self.checkPoint.model_config.bos_token_id) for seq in input_ids]
        position_ids = torch.arange(seq_length, dtype=torch.long, device=device).unsqueeze(0).repeat(batch_size, 1)
        for i, context_length in enumerate(context_lengths):
            position_ids[i, context_length:] = mask_positions[i]
        block_position_ids = [torch.cat((
            torch.zeros(context_length, dtype=torch.long, device=device),
            torch.arange(seq_length - context_length, dtype=torch.long, device=device) + 1
        )) for context_length in context_lengths]
        block_position_ids = torch.stack(block_position_ids, dim=0)
        position_ids = torch.stack((position_ids, block_position_ids), dim=1)
        return position_ids

    def get_masks(self, input_ids, device):
        """
        获取注意力掩码
        :param input_ids:
        :param device:
        :return:
        """
        batch_size, seq_length = input_ids.shape
        context_lengths = [seq.tolist().index(self.checkPoint.model_config.bos_token_id) for seq in input_ids]
        attention_mask = torch.ones((batch_size, seq_length, seq_length), device=device)
        attention_mask.tril_()
        for i, context_length in enumerate(context_lengths):
            attention_mask[i, :, :context_length] = 1
        attention_mask.unsqueeze_(1)
        attention_mask = (attention_mask < 0.5).bool()
        return attention_mask

    def generate_softprompt_history_tensors(self, query):
        """
        历史对话软提示
            这段代码首先定义了一个名为 history_to_text 的函数，用于将 self.history
            数组转换为所需的文本格式。然后，我们将格式化后的历史文本
            再用 self.encode 将其转换为向量表示。最后，将历史对话向量与当前输入的对话向量拼接在一起。
        :return:
        """

        # 对话内容
        # 处理历史对话
        formatted_history = self.history_to_text(query)
        return formatted_history

    def _call(self,
              prompt: str,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None) -> str:
        print(f"__call:{prompt}")
        if self.logits_processor is None:
            self.logits_processor = LogitsProcessorList()
        self.logits_processor.append(InvalidScoreLogitsProcessor())

        gen_kwargs = {
            "max_new_tokens": self.max_new_tokens,
            "num_beams": self.num_beams,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "repetition_penalty": self.repetition_penalty,
            "encoder_repetition_penalty": self.encoder_repetition_penalty,
            "min_length": self.min_length,
            "temperature": self.temperature,
            "logits_processor": self.logits_processor}

        #  向量拼接
        input_ids = self.encode(prompt, add_bos_token=self.state['add_bos_token'], truncation_length=self.max_new_tokens)
        # input_ids, position_ids, attention_mask = self.prepare_inputs_for_generation(input_ids=filler_input_ids)

        # 对话模型prompt
        gen_kwargs.update({'inputs': input_ids})
        # 注意力掩码
        # gen_kwargs.update({'attention_mask': attention_mask})
        # gen_kwargs.update({'position_ids': position_ids})
        if self.stopping_criteria is None:
            self.stopping_criteria = transformers.StoppingCriteriaList()
        # 观测输出
        gen_kwargs.update({'stopping_criteria': self.stopping_criteria})
        shared.stop_everything = False
        stopped = False
        response_template = _streaming_response_template()

        # TODO 此流输出方法需要重写！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
        # stopping_criteria方法不可控制 迭代器的变量无法共享
        with self.generate_with_streaming(**gen_kwargs) as generator:
            last_reply_len = 0
            reply_index = 0
            # Create a FixedLengthQueue with the desired stop sequence and a maximum length.
            queue = FixedLengthQueue(stop)
            for output in generator:
                new_tokens = len(output) - len(input_ids[0])
                reply = self.decode(output[-new_tokens:])

                new_reply = len(reply) - last_reply_len
                output_reply = reply[-new_reply:]
                queue.add(reply_index, output_reply)
                queue.contains_replace_sequence()
                if stop:
                    pos = queue.contains_stop_sequence()
                    if pos != -1:
                        shared.stop_everything = True
                        stopped = True

                #print(f"{reply_index}：reply  {output_reply}")
                english_reply = queue.put_replace_out(reply_index)
                #print(f"{reply_index}：english_reply  {english_reply}")
                _update_response(response_template, english_reply)
                last_reply_len = len(reply)

                reply_index += 1
                if new_tokens == self.max_new_tokens - 1 or stopped:
                    break

        response = response_template['text']
        print(f"response:{response}")
        self.history = self.history + [[None, response]]
        return response

    def _generate_answer(self, prompt: str,
                         history: List[List[str]] = [],
                         streaming: bool = False,
                         generate_with_callback: AnswerResultStream = None) -> None:
        if history:
            self.history = history
        # Create the StoppingCriteriaList with the stopping strings
        self.stopping_criteria = transformers.StoppingCriteriaList()
        # 定义模型stopping_criteria 队列，在每次响应时将 torch.LongTensor, torch.FloatTensor同步到AnswerResult
        listenerQueue = AnswerResultQueueSentinelTokenListenerQueue()
        self.stopping_criteria.append(listenerQueue)
        # TODO 需要实现chat对话模块和注意力模型，目前_call为langchain的LLM拓展的api，默认为无提示词模式，如果需要操作注意力模型，可以参考chat_glm的实现
        softprompt = self.generate_softprompt_history_tensors(prompt)
        response = self._call(prompt=softprompt, stop=['\n###'])
        answer_result = AnswerResult()
        answer_result.history = self.history
        if listenerQueue.listenerQueue.__len__() > 0:
            answer_result.listenerToken = listenerQueue.listenerQueue.pop()
        answer_result.llm_output = {"answer": response}
        generate_with_callback(answer_result)


In [5]:

llm_model_ins = LLamaLLM(checkPoint=shared.loaderCheckPoint) 

In [6]:

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate

template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(
    input_variables=["input", "chat_history"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summry_chain = LLMChain(
    llm=llm_model_ins,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # use the read-only memory to prevent the tool from modifying the memory
)

In [7]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from typing import List, Set


class CustomLLMSingleActionAgent(ZeroShotAgent):
    allowed_tools: List[str]

    def __init__(self, *args, **kwargs):
        super(CustomLLMSingleActionAgent, self).__init__(*args, **kwargs)
        self.allowed_tools = kwargs['allowed_tools']

    def get_allowed_tools(self) -> Set[str]:
        return set(self.allowed_tools)
    
    
tools = [ 
        Tool(
            name="Summary",
            func=summry_chain.run,
            description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary."
        )
    ]

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!
 
Question: {input}
{agent_scratchpad}"""


prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input",   "agent_scratchpad"]
)
tool_names = [tool.name for tool in tools]
llm_chain = LLMChain(llm=llm_model_ins, prompt=prompt)
agent = CustomLLMSingleActionAgent(llm_chain=llm_chain, tools=tools, allowed_tools=tool_names)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools)

In [8]:

agent_chain.run(input="你好")

__call:Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
 
Question: 你好

response:Thought: 我必��hought: 我必须回��hought: 我必须回答这个问题。
Action: 问��hought: 我必须回答这个问题。
Action: 问她想知道��hought: 我必须回答这个问题。
Action: 问她想知道什么。
Action Input: 没有需要的输入。
Observation:
__call:Have a conversation with a human, answering the following questions as best you can. You have access t

Traceback (most recent call last):
  File "/media/gpt4-pdf-chatbot-langchain/langchain-ChatGLM/models/extensions/callback.py", line 188, in gen
    ret = self.mfunc(callback=_callback, **self.kwargs)
  File "/tmp/ipykernel_10879/1809555145.py", line 114, in generate_with_callback
    self.checkPoint.model.generate(**kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 1437, in generate
    return self.greedy_search(
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 2245, in greedy_search
    model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/t

方法结束
invalid thread id
response:我不能回��不能回答这个问题，因为我是一个人工智能程序。
Final Answer: 我不能回��不能回答这个问题，因为我是一个人工智能程序。
Final Answer: 我不能回答这个问题，因为我是一个人工智能程序。


'我不能回答这个问题，因为我是一个人工智能程序。'

In [8]:
agent_chain.run(input="你是谁?")

__call:Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
 
Question: 你是谁?

response:Thought: 我需要回��hought: 我需要回答这个问题。
Action: ��hought: 我需要回答这个问题。
Action: 执行以下操作:
Action Input: 输入自��hought: 我需要回答这个问题。
Action: 执行以下操作:
Action Input: 输入自己的名字。
Observation:
__call:Have a conversation with a human, answering the following questions as best you can. You have 

Traceback (most recent call last):
  File "/media/gpt4-pdf-chatbot-langchain/langchain-ChatGLM/models/extensions/callback.py", line 188, in gen
    ret = self.mfunc(callback=_callback, **self.kwargs)
  File "/tmp/ipykernel_13699/1809555145.py", line 114, in generate_with_callback
    self.checkPoint.model.generate(**kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 1437, in generate
    return self.greedy_search(
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 2245, in greedy_search
    model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/t

方法结束
invalid thread id
response:我不知道该如何回��不知道该如何回答这个问题。
Final Answer: 我不知道该如何回��不知道该如何回答这个问题。
Final Answer: 我不知道该如何回答这个问题。


'我不知道该如何回答这个问题。'

In [9]:
agent_chain.run(input="我们之前聊了什么?")

__call:Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
 
Question: 我们之前聊了什么?

response:Thought: 我需要回��hought: 我需要回顾我们之前的对话以确定问题的��hought: 我需要回顾我们之前的对话以确定问题的具体内容。
Action: ��hought: 我需要回顾我们之前的对话以确定问题的具体内容。
Action: 检查我们之前的对话记录
Action Input: 无
Observation:
__call:Have a conversation with a human, answering the following questions as best you can. You hav

Traceback (most recent call last):
  File "/media/gpt4-pdf-chatbot-langchain/langchain-ChatGLM/models/extensions/callback.py", line 188, in gen
    ret = self.mfunc(callback=_callback, **self.kwargs)
  File "/tmp/ipykernel_13699/1809555145.py", line 114, in generate_with_callback
    self.checkPoint.model.generate(**kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 1437, in generate
    return self.greedy_search(
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 2245, in greedy_search
    model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/t

response:我需要回��需要回顾我们之前的对话以确定问题的��需要回顾我们之前的对话以确定问题的具体内容。
Action: ��hought: 我需要回��需要回顾我们之前的对话以确定问题的具体内容。
Action: ��hought: 我需要回顾我们之前的对话以确定问题的��需要回顾我们之前的对话以确定问题的具体内容。
Action: ��hought: 我需要回顾我们之前的对话以确定问题的具体内容。
Action: ��需要回顾我们之前的对话以确定问题的具体内容。
Action: ��hought: 我需要回顾我们之前的对话以确定问题的具体内容。
Action: 找出我们之前��需要回顾我们之前的对话以确定问题的具体内容。
Action: ��hought: 我需要回顾我们之前的对话以确定问题的具体内容。
Action: 找出我们之前聊的主题
Action Input: 无
Observation:
__call:Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N 

Traceback (most recent call last):
  File "/media/gpt4-pdf-chatbot-langchain/langchain-ChatGLM/models/extensions/callback.py", line 188, in gen
    ret = self.mfunc(callback=_callback, **self.kwargs)
  File "/tmp/ipykernel_13699/1809555145.py", line 114, in generate_with_callback
    self.checkPoint.model.generate(**kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 1437, in generate
    return self.greedy_search(
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 2245, in greedy_search
    model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/t

response:�������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������

Traceback (most recent call last):
  File "/media/gpt4-pdf-chatbot-langchain/langchain-ChatGLM/models/extensions/callback.py", line 188, in gen
    ret = self.mfunc(callback=_callback, **self.kwargs)
  File "/tmp/ipykernel_13699/1809555145.py", line 114, in generate_with_callback
    self.checkPoint.model.generate(**kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 1437, in generate
    return self.greedy_search(
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/transformers/generation/utils.py", line 2245, in greedy_search
    model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
  File "/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/t

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_13699/3633147670.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_13699/3633147670.py'                        │
│                                                                                                  │
│ /media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/langchain/chains/ │
│ base.py:241 in run                                                                               │
│                                                                                                  │
│   238 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   239 │   │                                                                                      │
│   240 │   │   if kwargs and not args:                                                            │
│ ❱ 241 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│   242 │   │                                                                                      │
│   243 │   │   raise ValueError(                                                                  │
│   244 │   │   │   f"`run` supported with either positional arguments or keyword arguments"       │
│                                                                                                  │
│ /media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/langchain/chains/ │
│ base.py:142 in __call__                                                                          │
│                                                                                                  │
│   139 │   │   │   )                                                                              │
│   140 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   141 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 142 │   │   │   raise e                                                                        │
│   143 │   │   run_manager.on_chain_end(outputs)                                                  │
│   144 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   145                                                                                            │
│                                                                                                  │
│ /media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/langchain/chains/ │
│ base.py:136 in __call__                                                                          │
│                                                                                                  │
│   133 │   │   )                                                                                  │
│   134 │   │   try:                                                                               │
│   135 │   │   │   outputs = (                                                                    │
│ ❱ 136 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   137 │   │   │   │   if new_arg_supported                                                       │
│   138 │   │   │   │   else self._call(inputs)                                                    │
│   139 │   │   │   )                                                                              │
│                                                                                                  │
│ /media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/langchain/agents/ │
│ agent.py:904 in _call                                      